### Overview over dataset

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
dataset = 'male_players.csv'
dataset_df = pd.read_csv(dataset)
print (dataset_df.head())

Check Correlations

In [ ]:
correlation_matrix = df.corr()

sns.set(style="white")
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()